Notebook to use HuggingFace chatbots via API using langchain

In [39]:


from dotenv import load_dotenv
load_dotenv()
import os

token = os.getenv("HF_ACCESS_INFERENCE_TOKEN")


In [40]:
# import libraries 
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

In [41]:
# initialize the LLM

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",  
    temperature=0.001,
    max_new_tokens=100
)





In [42]:
from typing import TypedDict, List, Dict, Any, Optional, Annotated
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage

# Define the state

class AgentState(TypedDict):
    question: Optional[str]
    last_ai_message: Optional[str]
    final_answer: Optional[str]
    messages: Annotated[list[Any], Any]

In [43]:
from langchain_tavily import TavilySearch

TAVILY_KEY = os.getenv("AGENT_TAVILY_API_KEY")

tavily_search = TavilySearch(
    tavily_api_key=TAVILY_KEY,
    max_results=2,
    topic="general",
    include_answer=True,
    # include_images=False,
    # include_image_descriptions=False,
    search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [44]:
def run_agent_for_question(question_dict: dict) -> AgentState:
    state: AgentState = {
        "question": question_dict["question"],
        "last_ai_message": None,
        "final_answer": None,
        "messages": []
    }

    # Paso 1: Guardar el mensaje del usuario
    user_msg = HumanMessage(content=state["question"])
    state["messages"].append(user_msg)

    # Paso 2: Hacer búsqueda con Tavily
    results = tavily_search.invoke({"query": state["question"]})

    # Simula razonamiento paso a paso con el agente
    answer_parts = []
    for i, result in enumerate(results.get("results", [])):
        content = result.get("content", "")
        ai_msg = AIMessage(content=f"Step {i+1}: {content}")
        state["messages"].append(ai_msg)
        answer_parts.append(content)

        # Puedes decidir cuándo parar, por ahora asumimos que con 2 resultados basta
        if len(answer_parts) >= 2:
            break

    # Paso final: consolidar respuesta
    final = " ".join(answer_parts)
    state["final_answer"] = final
    state["last_ai_message"] = final
    state["messages"].append(AIMessage(content=f"Final answer: {final}"))

    return state

In [45]:
questions = [
    {"question": "What is LangChain used for?"},
    {"question": "Who created Python?"}
]

for q in questions:
    result = run_agent_for_question(q)
    print("== Question:", result["question"])
    print("== Final Answer:", result["final_answer"])
    print("== Messages:")
    for msg in result["messages"]:
        print(f"  [{msg.type}] {msg.content}")
    print()


== Question: What is LangChain used for?
== Final Answer: Examples and use cases for LangChain. The LLM-based applications LangChain is capable of building can be applied to multiple advanced use cases within various industries and vertical markets, such as the following: Customer service chatbots. LangChain enables advanced chat applications to handle complex questions and user transactions. Build a simple LLM application with chat models and prompt templates How to migrate from legacy LangChain agents to LangGraph How to use chat models to call tools How to pass tool outputs to chat models langchain: Chains, agents, and retrieval strategies that make up an application's cognitive architecture. However, these guides will help you quickly accomplish common tasks using chat models, vector stores, and other common LangChain components. LangChain is part of a rich ecosystem of tools that integrate with our framework and build on top of it. If you're looking to get up and running quickly w